# 2021/04/22 - 2

In [ ]:
# まずは全てインポート
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, IBMQ, execute
from qiskit.tools.monitor import job_monitor
from qiskit.providers.ibmq import least_busy
# このワークブック独自のモジュール
from utils.dynamics import plot_heisenberg_spins

## 実習：ハイゼンベルグモデルの時間発展

### 1トロッターステップずつ長い回路をM個作る

In [ ]:
n = 5
M = 10
omegadt = 0.1

circuits = []

circuit = QuantumCircuit(n)

# Bit 0 in state 1/sqrt(2)(|0> + |1>)
circuit.h(0)

# Loop over Trotter steps
for istep in range(M):

    # Loop over spins
    for j in range(n - 1):
        # ZZ
        circuit.cx(j, j + 1)
        circuit.rz(-omegadt, j + 1)
        circuit.cx(j, j + 1)

        # XX
        circuit.h(j)
        circuit.h(j + 1)
        circuit.cx(j, j + 1)
        circuit.rz(-omegadt, j + 1)
        circuit.cx(j, j + 1)
        circuit.h(j)
        circuit.h(j + 1)

        # YY
        circuit.p(-np.pi / 2., j)
        circuit.p(-np.pi / 2., j + 1)
        circuit.h(j)
        circuit.h(j + 1)
        circuit.cx(j, j + 1)
        circuit.rz(-omegadt, j + 1)
        circuit.cx(j, j + 1)
        circuit.h(j)
        circuit.h(j + 1)
        circuit.p(np.pi / 2., j)
        circuit.p(np.pi / 2., j + 1)

    # Save a copy of the circuit at this point
    # measure_all(inplace=False) creates a copy of the circuit with a full measurement at the end
    circuits.append(circuit.measure_all(inplace=False))
    
print('{} circuits created'.format(len(circuits)))

### 量子回路シミュレーターで実行し、各ビットにおけるZ方向スピンの期待値をプロットする

プロット用の関数は既に定義してある。関数はジョブの実行結果、系のスピンの数、初期状態、ステップ間隔を引数にとる。

In [ ]:
# Define the initial statevector
initial_state = np.zeros(2 ** n, dtype=np.complex128)
initial_state[0:2] = np.sqrt(0.5)

shots = 100000

qasm_simulator = Aer.get_backend('qasm_simulator')

sim_job = execute(circuits, qasm_simulator, shots=shots)
sim_counts_list = sim_job.result().get_counts()
   
plot_heisenberg_spins(sim_counts_list, n, initial_state, omegadt, add_theory_curve=True)

ビット0でのスピンの不整合が徐々に他のビットに伝搬していく様子が観察できる。

プロット中の実線はハミルトニアンの対角化を数値的に行って計算した数値厳密解。トロッター分解による解が少しずつ厳密解からずれていくのがわかる。`omegadt`を小さく、`M`を大きく取り直したらずれは小さくなるだろうか？

### 実機でも同様に実行する

QV32（比較的良い性能）の5量子ビットマシンで確認してみましょう。

In [ ]:
# もしもload_account()でエラーが出たら、2021_04_15_1.ipynbを参考にIBMQトークンを一度保存してください
#IBMQ.save_account('__paste_your_token_here__')
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

# Find the least busy backend with >= 5 bits and QV >= 32
backend_filter = lambda b: (not b.configuration().simulator) and b.configuration().n_qubits >= n and b.configuration().quantum_volume >= 32 and b.status().operational
backend = least_busy(provider.backends(filters=backend_filter))

job = execute(circuits, backend, shots=8192, optimization_level=3)

job_monitor(job, interval=2)

counts_list = job.result().get_counts()

In [ ]:
plot_heisenberg_spins(counts_list, n, initial_state, omegadt)